## Data preparation of four scenarios for the full alignment case

In [109]:
import numpy as np
import librosa as lb
import soundfile as sf
from pathlib import Path
import matplotlib.pyplot as plt

Before we generate 4 different scenarios, we first define a function `splice_audio`.

In [110]:
def splice_audio(audio, start_sec, end_sec, sr):
    return audio[int(start_sec * sr): int(end_sec * sr)]

current_path = Path('.')
base_path = Path('/home/hjamal/ttmp/E207Project')

Define the pathway

In [111]:
train_data_path = base_path / 'train_data'
op17no4_1981_path = train_data_path / 'Chopin_Op017No4_Ashkenazy-1981_pid9058-13.wav'
op17no4_1995_path = train_data_path / 'Chopin_Op017No4_Block-1995_pid9064-04.wav'
op63no3_path = train_data_path / 'Chopin_Op063No3_Morozova-2008_pid610633-04.wav'

Load the audio file with `librosa`

In [112]:
op17no4_1981, sr_1 = lb.load(op17no4_1981_path)
op17no4_1995, sr_2 = lb.load(op17no4_1995_path)
op63no3_2008, sr_3 = lb.load(op63no3_path)

### Scenario I: Matching
- In this case, we use 30 seconds of `Op17No4_1981` and the full sequence of `Op17No4_1995`
- In this case, we align a subsequence of a song with a different full recording of the same song
- Recording 1 will be the shorter sequence in each scenario

if the sampling rates of Op17_1981 and Op17_1995 are not the same

In [113]:
if sr_1 != sr_2:
    op17no4_1995 = librosa.resample(op17no4_1995, orig_sr=sr_2, target_sr=sr_1)

In [114]:
soundfiles_path = current_path / 'subsequence_soundfiles/'
scenario1_destination = soundfiles_path / 'scenario_1/'
sf.write(scenario1_destination / 'partial_align_matching_recording1.wav', splice_audio(op17no4_1981, 10, 40, sr_1), sr_1)
sf.write(scenario1_destination / 'partial_align_matching_recording2.wav', op17no4_1995, sr_1)

### Scenario II: Non-matching
- In this case, we use `Op17No4_1981` and `Op63No3_2008`
- In this case, we align a subsequence of the first recording with a complete recording of different Mazurka

if the sampling rates of Op17_1981 and Op17_1995 are not the same

In [115]:
if sr_1 != sr_3:
    op63no3_2008 = librosa.resample(op63no3_2008, orig_sr=sr_3, target_sr=sr_1)

In [116]:
scenario2_destination = soundfiles_path / 'scenario_2/'
sf.write(scenario2_destination / 'partial_align_non_matching_recording1.wav', splice_audio(op17no4_1981, 10, 40, sr_1), sr_1)
sf.write(scenario2_destination / 'partial_align_non_matching_recording2.wav', op63no3_2008, sr_1)

### Scenario III: Matching with a mismatching region in the middle
- In this case, we use `Op17No4_1981`, `Op17No4_1995` and `Op63No3_2008`
- Recording 1: 30s of `Op17No4_1995`, where a 10s chunk in the middle is spliced out and replaced with a 10s chunk from `Op63No3_2008`
- Recording 2: `Op17No4_1981`

In [117]:
def add_chunk(chunk_sec, main_audio, chunk_audio, sr=22050):
    half_len_main = len(main_audio)//2
    half_len_chunk = len(chunk_audio)//2

    # Calculate the start sample of the 30s chunk in the middle of main audio & chunk audio
    start_main = half_len_main - (chunk_sec * sr // 2)
    start_chunk = half_len_chunk - (chunk_sec * sr // 2)

    # Splice out the 30s chunk from main audio and replace with a chunk from chunk audio
    samples = chunk_sec * sr
    replacement = chunk_audio[start_chunk : start_chunk+samples]
    recording_b = np.concatenate([
        main_audio[:start_main],
        replacement,
        main_audio[start_main+samples:]
    ])

    return recording_b

Compute the Recording B

In [118]:
recording_1 = add_chunk(10, splice_audio(op17no4_1995, 10, 40, sr_1), splice_audio(op63no3_2008, 20, 30, sr_1))

In [119]:
scenario3_destination = soundfiles_path / 'scenario_3/'

sf.write(scenario3_destination / 'partial_align_mid_mismatch_recording1.wav', recording_1, sr_1)
sf.write(scenario3_destination / 'partial_align_mid_mismatch_recording2.wav', op17no4_1981, sr_2)

### Scenario IV: Non-Matching with a matching region in the middle
- In this case, we use `Op17No4_1981`, `Op17No4_1995` and `Op63No3_2008`
- Recording 1: 30s of `Op63No3_2008`, where a 10s chunk in the middle is spliced out and replaced with a 10s chunk from `Op17No4_1995`
- Recording 2: `Op17No4_1981`


In [120]:
recording_1 = add_chunk(10, splice_audio(op63no3_2008, 10, 40, sr_1), splice_audio(op17no4_1995, 20, 30, sr_1))

In [121]:
scenario4_destination = soundfiles_path / 'scenario_4/'

sf.write(scenario4_destination / 'partial_align_mid_match_recording1.wav', recording_1, sr_1)
sf.write(scenario4_destination / 'partial_align_mid_match_recording2.wav', op17no4_1981, sr_2)